# RL basics

Термины и понятия:

- агент/среда
- наблюдение $o$ / состояние $s$
- действие $a$, стратегия $\pi: \pi(s) \rightarrow a$ функция перехода $T: T(s, a) \rightarrow s'$
- вознаграждение $r$, ф-я вознаграждений $R: R(s, a) \rightarrow r$
- цикл взаимодействия, траектория $\tau: (s_0, a_0, r_0, s_1, a_1, r_1, ..., s_T, a_T, r_T)$, эпизод
- отдача $G$, подсчет отдачи, средняя[/ожидаемая] отдача $\mathbb{E}[G]$

In [15]:
try:
    import google.colab
    COLAB = True
except ModuleNotFoundError:
    COLAB = False
    pass

if COLAB:
    !pip -q install "gymnasium[classic-control, atari, accept-rom-license]"
    !pip -q install piglet
    !pip -q install imageio_ffmpeg
    !pip -q install moviepy==1.0.3

In [16]:
import glob
import io
import base64
import gymnasium as gym
import numpy as np
from IPython import display as ipythondisplay
from IPython.display import HTML
import matplotlib.pyplot as plt
%matplotlib inline

## Agent, environment

<img src=https://gymnasium.farama.org/_images/lunar_lander.gif caption="lunar lander" width="150" height="50"><img src=https://gymnasium.farama.org/_images/mountain_car.gif caption="mountain car" width="150" height="50">
<img src=https://gymnasium.farama.org/_images/cliff_walking.gif caption="cliff walking" width="300" height="50">
<img src=https://ale.farama.org/_images/montezuma_revenge.gif caption="montezuma revenge" width="150" height="100">
<img src=https://github.com/danijar/crafter/raw/main/media/video.gif caption="crafter" width="150" height="100">
<img src=https://camo.githubusercontent.com/6df2ca438d8fe8aa7a132b859315147818c54af608f8609320c3c20e938acf48/68747470733a2f2f6d656469612e67697068792e636f6d2f6d656469612f344e78376759694d394e44724d724d616f372f67697068792e676966 caption="malmo minecraft" width="150" height="100">
<img src=https://images.ctfassets.net/kftzwdyauwt9/e0c0947f-1a44-4528-4a41450a9f0a/2d0e85871d58d02dbe01b2469d693d4a/table-03.gif caption="roboschool" width="150" height="100">
<img src=https://raw.githubusercontent.com/Tviskaron/mipt/master/2019/RL/02/mdp.png caption="Марковский процесс принятия решений" width="150" height="100">
<img src=https://minigrid.farama.org/_images/DoorKeyEnv.gif caption="minigrid" width="120" height="120">

## Observation, state

TODO:
- добавить примеры наблюдений/состояний (числа, векторы, картинки)
- интуитивное объяснение различия, положить пока, что наблюдение = состояние
- пространство состояний


В каждый момент времени среда имеет некоторое внутреннее состояние. Здесь слово "состояние" я употребил скорее в интуитивном понимании, чтобы обозначить, что среда изменчива (иначе какой смысл с ней взаимодействовать, если ничего не меняется). В обучении с подкреплением под термином состояние $s$ (или $s_t$, где $t$ — текущее время) подразумевают либо абстрактно информацию о "состоянии" среды, либо ее явное представление в виде данных, достаточные для полного описания "состояния". *NB: Здесь можно провести аналогию с компьютерными играми — файл сохранения игры как раз содержит информацию о "состоянии" мира игры, чтобы в будущем можно было продолжить с текущей точки, так что данные этого файла в целом можно с некоторой натяжкой считать состоянием (с натяжкой, потому что редко когда в сложных играх файлы сохранения содержат прямо вот всю информацию, так что после перезагрузки вы получите не совсем точную копию). При этом обычно подразумевается, что состояние не содержит в себе ничего лишнего, то есть это **минимальный** набор информации.*

Наблюдением $o$ называют то, что агент "видит" о текущем состоянии среды. Это не обязательно зрение, а вообще вся доступная ему информация (условно, со всех его органов чувств).

В общем случае наблюдение: кортеж/словарь многомерных векторов чисел.

In [17]:
print(gym.make("CartPole-v0").reset()[0].shape)
print(gym.make("MountainCar-v0").reset()[0].shape)

(4,)
(2,)


## Action, policy, transition function

Рассмотрим следующие MDP:

- A: <img src=https://i.ibb.co/mrCMVZLQ/mdp-a.png caption="A" width="400" height="100">
- B: <img src=https://i.ibb.co/GQ2tVtjC/mdp-b.png caption="B" width="400" height="100">

Links to all:
[A](https://i.ibb.co/mrCMVZLQ/mdp-a.png)
[B](https://i.ibb.co/GQ2tVtjC/mdp-b.png)
[C](https://i.ibb.co/Jj9LYHjP/mdp-c.png)
[D](https://i.ibb.co/Y47Mr83b/mdp-d.png)
[E](https://i.ibb.co/Kjt1Xhmf/mdp-e.png)

Давайте явно запишем пространства состояний $S$ и действий $A$, а также функцию перехода $T$ среды.

In [18]:
A_states = set(range(3))
A_actions = set(range(1))

B_states = set(range(4))
B_actions = set(range(3))

C_states = set(range(4))
C_actions = set(range(2))

print(f'{A_states=} | {A_actions=}')

A_T = {
    (0, 0): 1,
    (1, 0): 2,
    (2, 0): 2
}

B_T = {
    (0, 0): 1,
    (0, 1): 2,
    (0, 2): 3,
    (1, 0): 1, 
    (1, 1): 1,
    (1, 2): 1,
    (2, 0): 2,
    (2, 1): 2,
    (2, 2): 2,
    (3, 0): 3,
    (3, 1): 3,
    (3, 2): 3
}

C_T = {
    (0, 0): 1,
    (0, 1): 2,
    (1, 0): 1,
    (1, 1): 3,
    (2, 0): 3, 
    (2, 1): 2,
    (3, 0): 3, 
    (3, 1): 3
}
print(f'Transition function {A_T=}')

A_mdp = A_states, A_actions, A_T
B_mdp = B_states, B_actions, B_T
C_mdp = C_states, C_actions, C_T

A_states={0, 1, 2} | A_actions={0}
Transition function A_T={(0, 0): 1, (1, 0): 2, (2, 0): 2}


Попробуйте записать функцию перехода в матричном виде:

In [19]:
A_init_distr = np.array([1.0, 0.0, 0.0])
B_init_distr = np.array([1.0, 0.0, 0.0, 0.0])
C_init_distr = np.array([1.0, 0.0, 0.0, 0.0])

A_matrix = np.array([[0.0, 1.0, 0.0],
                     [0.0, 0.0, 1.0],
                     [0.0, 0.0, 1.0]])

B_matrix = np.array([[0.0, 0.2, 0.5, 0.3],
                     [0.0, 1.0, 0.0, 0.0],
                     [0.0, 0.0, 1.0, 0.0],
                     [0.0, 0.0, 0.0, 1.0]])

C_matrix = np.array([[0.0, 0.5, 0.5, 0.0],
                     [0.0, 0.5, 0.0, 0.5],
                     [0.0, 0.0, 0.5, 0.5],
                     [0.0, 0.0, 0.0, 1.0]])

def get_prob_after_n_steps_for_all_states(init_distr, P_matrix, N):
    return init_distr @ np.pow(P_matrix, N)
    
def get_prob_after_n_steps_for_one_state(init_distr, P_matrix, N, state):
    vec = get_prob_after_n_steps_for_all_states(init_distr, P_matrix, N)
    return vec[state]

Как получить вероятность нахождения агента в состоянии (1) через N шагов? Что происходит с вероятностями нахождения в состояниях при $N \rightarrow \infty$

In [20]:
# получение вероятности для состояния 1 для схемы А, N = 5 например
prob_A_one = get_prob_after_n_steps_for_one_state(A_init_distr, A_matrix, 5, 1)

# получение вероятности для всех состояний для схемы А, N = 100 например
prob_A_all = get_prob_after_n_steps_for_all_states(A_init_distr, A_matrix, 100)

# получение вероятности для состояния 1 для схемы B, N = 5 например
prob_B_one = get_prob_after_n_steps_for_one_state(B_init_distr, B_matrix, 5, 1)

# получение вероятности для всех состояний для схемы C, N = 100 например
prob_B_all = get_prob_after_n_steps_for_all_states(B_init_distr, B_matrix, 100)

# получение вероятности для состояния 1 для схемы C, N = 5 например
prob_C_one = get_prob_after_n_steps_for_one_state(C_init_distr, C_matrix, 5, 1)

# получение вероятности для всех состояний для схемы C, N = 100 например
prob_C_all = get_prob_after_n_steps_for_all_states(C_init_distr, C_matrix, 100)

print(f"A schema: one - {prob_A_one}, all - {prob_A_all}")
print(f"B schema: one - {prob_B_one}, all - {prob_B_all}")
print(f"C schema: one - {prob_C_one}, all - {prob_C_all}")

A schema: one - 1.0, all - [0. 1. 0.]
B schema: one - 0.0003200000000000001, all - [0.00000000e+00 1.26765060e-70 7.88860905e-31 5.15377521e-53]
C schema: one - 0.03125, all - [0.00000000e+00 7.88860905e-31 7.88860905e-31 0.00000000e+00]


Задайте еще несколько MDP:

- C: <img src=https://i.ibb.co/Jj9LYHjP/mdp-c.png caption="C" width="400" height="100">

Давайте попробуем задать двух агентов: случайного и оптимального (для каждой среды свой).

In [21]:
class Agent:
    def __init__(self, actions):
        self.rng = np.random.default_rng()
        self.actions = np.array(list(actions))

    def act(self, state):
        return self.rng.integers(len(self.actions))

В качестве дополнения, запишите стратегию агента

In [22]:
import random

class SmartAgentA:
    def __init__(self, actions):
        self.actions = np.array(list(actions))

    def act(self, state):
        return 0

class SmartAgentB:
    def __init__(self, actions):
        self.actions = np.array(list(actions))

    def act(self, state):
        return random.choice([0, 1, 2])

class SmartAgentC:
    def __init__(self, actions):
        self.actions = np.array(list(actions))

    def act(self, state):
        if state == 0:
            return random.choice([0, 1])
        if state == 1:
            return 1
        if state == 2:
            return 0
        return random.choice([0, 1])

## Reward, reward function

Теперь добавим произвольную функцию вознаграждения. Например, для A:

In [23]:
R_A = {
    (0, 0): -0.1,
    (1, 0): 1.0,
    (2, 0): 0.0
}

R_B = {
    (0, 0): 1.0,
    (0, 1): 1.0,
    (0, 2): 1.0,
    (1, 0): 0.0, 
    (1, 1): 0.0,
    (1, 2): 0.0,
    (2, 0): 0.0, 
    (2, 1): 0.0,
    (2, 2): 0.0,
    (3, 0): 0.0, 
    (3, 1): 0.0,
    (3, 2): 0.0,
}

R_C = {
    (0, 0): 1.0,
    (0, 1): 1.0,
    (1, 0): 0.0,
    (1, 1): 1.0,
    (2, 0): 1.0,
    (2, 1): 0.0,
    (3, 0): 0.0,
    (3, 1): 0.0,
}

print(R_A)

A_mdp = *A_mdp, R_A
B_mdp = *B_mdp, R_B
C_mdp = *C_mdp, R_C
print(A_mdp)

{(0, 0): -0.1, (1, 0): 1.0, (2, 0): 0.0}
({0, 1, 2}, {0}, {(0, 0): 1, (1, 0): 2, (2, 0): 2}, {(0, 0): -0.1, (1, 0): 1.0, (2, 0): 0.0})


## Interaction loop, trajectory, termination, truncation, episode

Общий цикл взаимодействия в рамках эпизода:
1. Инициализировать среду: $s \leftarrow \text{env.init()}$
2. Цикл:
    - выбрать действие: $a \leftarrow \pi(s)$
    - получить ответ от среды: $s, r, d \leftarrow \text{env.next(a)}$
    - если $d == \text{True}$, выйти из цикла

In [24]:
def run_episode(mdp, agentClass, term_state):
    states, actions, T, R = mdp
    agent = agentClass(actions)

    s = 0
    tau = []
    for _ in range(5):
        if s == term_state:
            break
        a = agent.act(s)
        s_next = T[(s, a)]
        r = R[(s, a)]

        tau.append((s, a, r))
        s = s_next

    return tau

run_episode(A_mdp, Agent, 2)

[(0, np.int64(0), -0.1), (1, np.int64(0), 1.0)]

In [25]:
run_episode(A_mdp, SmartAgentA, 2)

[(0, 0, -0.1), (1, 0, 1.0)]

In [26]:
run_episode(B_mdp, SmartAgentB, 3)

[(0, 2, 1.0)]

In [27]:
run_episode(C_mdp, SmartAgentC, 3)

[(0, 0, 1.0), (1, 1, 1.0)]

Termination — означает окончание эпизода, когда достигнуто терминальное состояние. Является частью задания среды.

Truncation — означает окончание эпизода, когда достигнут лимит по числу шагов (=времени). Обычно является внешне заданным параметром для удобства обучения.

Пока не будем вводить truncation, но поддержим termination: расширьте определение среды информацией о терминальных состояниях для всех описанных ранее сред. Сгенерируйте по несколько случайных траекторий для каждой среды.

### Return, expected return

Наиболее важная метрика оценки качества работы агента: отдача.

Отдача: $G(s_t) = \sum_{i=t}^T r_i$

Обычно также вводят параметр $\gamma \in [0, 1]$, дисконтирующий будущие вознаграждения. А еще, тк отдача может меняться от запуска к запуску благодаря вероятностным процессам, нас интересует отдача в среднем — ожидаемая отдача:

$$\hat{G}(s_t) = \mathbb{E} [ \sum_{i=t}^T \gamma^{i-t} r_i ]$$

Именно ее и оптимизируют в RL.

Давайте научимся считать отдачу для состояний по траектории и считать среднюю отдачу.

In [29]:
N = 10

# for A
a_array = list()
for state in range(3):
    G_A = 0.0
    for i in range(N):
        traj = run_episode(A_mdp, SmartAgentA, state)
        sum_delta = 1.0
        delta = 0.95
        for s, a, r in traj:
            G_A += sum_delta * r
            sum_delta *= delta
    a_array.append(G_A / N)

#for B
b_array = list()
for state in range(4):
    G_B = 0.0
    for i in range(N):
        traj = run_episode(B_mdp, SmartAgentB, state)
        sum_delta = 1.0
        delta = 0.95
        for s, a, r in traj:
            G_B += sum_delta * r
            sum_delta *= delta
    b_array.append(G_B / N)

#for C
c_array = list()
for state in range(4):
    G_C = 0.0
    for i in range(N):
        traj = run_episode(C_mdp, SmartAgentC, state)
        sum_delta = 1.0
        delta = 0.95
        for s, a, r in traj:
            G_C += sum_delta * r
            sum_delta *= delta
    c_array.append(G_C / N)

print(f"a_array: {a_array}")
print(f"b_array: {b_array}")
print(f"c_array: {c_array}")

a_array: [0.0, -0.09999999999999999, 0.8500000000000002]
b_array: [0.0, 1.0, 1.0, 1.0]
c_array: [0.0, 1.6649999999999998, 1.6649999999999998, 1.9499999999999997]
